In [14]:
import pandas
import csv
import glob
import os
import sklearn

In [15]:
folder = os.path.normpath('C:/Users/daveysa1/Documents/GitHub/Random')
file = 'Data.csv'
filepath = os.path.join(folder,file)

In [16]:
df = pandas.read_csv(filepath)

In [59]:
df.shape 

(3689, 3)

In [61]:
# Split into Train and Test sets
df_train = df.ix[:round(len(df)*0.75), 'Description':'Category'] # 3/4 of the rows, description and category columns 
df_test = df.ix[round(len(df)*0.75)+1:, 'Description':'Category'] # 1/4 of the rows, description and category columns 

In [66]:
# check that our indexing worked
print('total:', len(df))
print('train:', len(df_train))
print('test:', len(df_test))
print('combined:', len(df_train) + len(df_test))
if len(df) == len(df_train) + len(df_test):
    print('>>> OK!')
else:
    print('>>> ERROR!')

total: 3689
train: 2768
test: 921
combined: 3689
>>> OK!


Looks like all text has to be tokenised before it can be useful.
http://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html suggests using Bag of Words to tokenize and look at frequency in order to determine classification. Transaction descriptions have few and relatively unique words, so Bag of Words should give precise classification.

look up principal component analysis / feature reduction.

In [68]:
from sklearn.feature_extraction.text import CountVectorizer
#from sklearn.feature_extraction.text import HashingVectorizer

# instantiate the vectorizer
vectoriser = CountVectorizer()

In [83]:
# Vectorizers are stateless, so fit() does nothing; you need to fit_transform.
# fit_transform() turns the sequence (transaction descriptions) into a matrix of words

# transform the whole set of Descriptions. The whole set because we are not training at this point, just wrangling.
# Words in each description are tokenised during the transform.
df_train_xform = vectoriser.fit_transform(df_train.ix[:,'Description'])
df_test_xform = vectoriser.transform(df_test.ix[:,'Description'])
new_descriptions = ['MYER MORLEY','COLES Bassendean','JB HIFI','Caltex Sydney','Liquorland Kalgoorlie']
manual_test = vectoriser.transform(new_descriptions)

In [84]:
print('top of training set:')
print(df_train.ix[1:2,:])
print()
print('transformed training set array:')
print(df_train_xform[:2].toarray())
print()

print('transformed TRAINing set shape:')
print(df_train_xform.shape)
print()
print('transformed TESTing set shape:')
print(df_test_xform.shape)
print()
print('transformed MANUAL PREDICT set shape:')
print(manual_test.shape)
print()

top of training set:
                               Description           Category
1                      DEBIT INT TO 29 JUN  Mortgage Interest
2  HOUSE IN MORLEY          MORLEY      WA         Renovation

transformed training set array:
[[0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]]

transformed TRAINing set shape:
(2768, 1957)

transformed TESTing set shape:
(921, 1957)

transformed MANUAL PREDICT set shape:
(5, 1957)



In [85]:
from sklearn.naive_bayes import MultinomialNB

# the classifier is not instantiated - all static
# it is fit() to the transformed training set, linked to the relevant Categories
clf = MultinomialNB().fit(df_train_xform, df_train.ix[:, 'Category'])

In [91]:
predicted = clf.predict(X_new_counts)
for description, category in zip(new_descriptions, predicted):
    print('%r => %s' % (description, category))

'MYER MORLEY' => Renovation
'COLES Bassendean' => Living and Bills
'JB HIFI' => Renovation
'Caltex Sydney' => Living and Bills
'Liquorland Kalgoorlie' => Living and Bills


The Hashing version

In [63]:
# Count the number of distinct words in the transaction descriptions
# This is needed for the n_features parameter of the HashingVectorizer
from collections import Counter
c = Counter()
for desc in df_train.ix[:, 'Description']:
    c.update(desc.lower().split())

len(c)

1983

In [43]:
# # instantiate the vectorizer
# vectoriser = HashingVectorizer(#input='content', # default: we're passing data directly rather than asking to analyze 1..n files
#                                #encoding='utf-8', # default
#                                #decode_error='strict', # default: raise UnicodeDecodeError if a sketchy char is found
#                                #strip_accents=None, # defaul: we expect transaction descriptions to be ascii anyway
#                                #lowercase=True, # default
#                                preprocessor=None,  # default
#                                tokenizer=None,  # default
#                                stop_words=None, # merchant names will have useful 'stop-words' that we want to keep
#                                #token_pattern='(?u)\b\w\w+\b', # default: keep the regexp
#                                ngram_range=(1, 1), 
#                                analyzer='word', # hash the words, not the individual characters
#                                n_features=10000, # we've counted the dustinct words and keeping all as features
#                                binary=False, 
#                                norm='l2', 
#                                non_negative=False, 
#                                #dtype=<class 'numpy.float64'>
#                               )
# # HashingVectorizers are stateless, so fit() does nothing; you need to fit_transform.
# # fit_transform() turns the sequence (transaction descriptions) into a matrix of words

# # transform the whole set of Descriptions. The whole set because we are not training at this point, just wrangling.
# df_xform = vectoriser.fit_transform(df.ix[:,'Description'])

In [19]:
#from sklearn import tree
training_set = df_xform.ix[:round(len(df_xform)*0.75), 'Description':'Category'] # 3/4 of the rows, description and category columns 
testing_set = df_xform.ix[round(len(df_xform)*0.75)+1:, 'Description'] # The other 1/4 of the rows, description column only

In [27]:
clf = tree.DecisionTreeClassifier()
clf = clf.fit(training_set, testing_set)
# throws exception because it is expecting floats rather than strings. 'Front Bathroom' is used at the bottom boundary of training_set and the top boundary of testing_set.
# pd.getdummies() will convert a list of features into a separate 1/0 vector for each, but this won't solve the problem of using string descriptions

ValueError: could not convert string to float: 'Front Bathroom'